In [1]:
import numpy as np
import pandas as pd


In [2]:
data = pd.read_parquet('../data/cleaned_data/data_with_all_weather.parquet')

Cancellation code

| Code | Description          |
|------|----------------------|
| A    | Carrier              |
| B    | Weather              |
| C    | National Air System  |
| D    | Security             |

In [16]:
# Flights with cancellation code are all cancelled
assert not (data[data['CANCELLATION_CODE'].notna()]['CANCELLED'] == 0).sum()
# Cancelled flights all have cancellation code
assert not data[data['CANCELLED'] == 1]['CANCELLATION_CODE'].isna().sum()
print('Cancellation proportion:', f"{round((data['CANCELLED'].sum() / len(data)) * 100, 3)}%")

Cancellation proportion: 1.943%


In [25]:
cancellation_info = data['CANCELLATION_CODE'].value_counts().reset_index()
cancellation_info.columns = ['Cancellation_Code', 'Count']
cancellation_info['Proportion'] = round((cancellation_info['Count'] / len(data)) * 100, 3)
cancellation_info

,Cancellation_Code,Count,Proportion
0,B,121900,1.114
1,A,71848,0.657
2,C,16659,0.152
3,D,2139,0.020


In [26]:
cancelled_weather = data[data['CANCELLATION_CODE'] == 'B']

In [27]:
data['wxcodes'].notna().sum()

1650182